# Commutativity checker for fermionic excitations in second quantization

This code asks for user input. The input is a sum of single and double excitation terms in the form:

`a^ : creation operator on qubit a`

`a  : annihilation operator on qubit a`

where `a` is the index of the qubit, i.e. 0, 1, 2, .....

In [36]:
import numpy as np
from itertools import combinations
from qiskit.quantum_info import SparsePauliOp

def get_single_jw_op(index, is_creation, n_qubits):
    """
    Creates a single creation/annihilation operator (a_p or a_p^dag).
    """
    # 1. Setup strings
    full_identity = ['I'] * n_qubits
    
    # 2. X/Y part at 'index' (Little-Endian: rightmost is 0)
    pos = n_qubits - 1 - index
    
    x_list = full_identity[:]
    x_list[pos] = 'X'
    
    y_list = full_identity[:]
    y_list[pos] = 'Y'
    
    # 3. Z-string (Parity) on 0 to index-1
    z_list = full_identity[:]
    if index > 0:
        for j in range(index):
            z_pos = n_qubits - 1 - j
            z_list[z_pos] = 'Z'
            
    # 4. Create Operators
    # coeff for Y: -0.5j (creation), +0.5j (annihilation)
    y_coeff = -0.5j if is_creation else 0.5j
    
    op_x = SparsePauliOp("".join(x_list), coeffs=[0.5])
    op_y = SparsePauliOp("".join(y_list), coeffs=[y_coeff])
    op_z = SparsePauliOp("".join(z_list), coeffs=[1.0])
    
    # (X + iY) * Z_string
    return (op_x + op_y) @ op_z

def robust_parse(input_str, n_qubits):
    """
    Robustly parses sums vs products.
    """
    print(f"\n--- Parsing Input: '{input_str}' ---")
    
    # 1. Normalize the string to handle "1^0+3^2" vs "1^ 0 + 3^ 2"
    norm_str = input_str.replace('+', ' + ')
    
    # 2. Split into ADDITIVE terms
    raw_sum_terms = norm_str.split('+')
    raw_sum_terms = [t.strip() for t in raw_sum_terms if t.strip()]
    
    if len(raw_sum_terms) > 1:
        print(f"Structure: SUM of {len(raw_sum_terms)} distinct terms.")
    else:
        print(f"Structure: SINGLE TERM (Product sequence).")
        
    total_T = None
    
    # 3. Process each additive term
    for i, term_str in enumerate(raw_sum_terms):
        # Split into MULTIPLICATIVE factors (e.g. "1^" "0")
        factors = term_str.split()
        term_product = None
        
        desc = []
        for factor in factors:
            is_creation = '^' in factor
            try:
                idx = int(factor.replace('^', ''))
            except ValueError:
                print(f"Error: Invalid index '{factor}'")
                return None
            
            op = get_single_jw_op(idx, is_creation, n_qubits)
            
            # Accumulate Product
            if term_product is None:
                term_product = op
            else:
                term_product = term_product @ op
                
            desc.append(f"a_{idx}{'^' if is_creation else ''}")
            
        print(f"  - Term {i+1}: {' '.join(desc)}")
        
        # Accumulate Sum
        if total_T is None:
            total_T = term_product
        else:
            total_T = total_T + term_product
            
    return total_T.simplify()

def check_commutativity_of_generator():
    # 1. Input
    try:
        n_qubits = int(input("Enter system size (qubits) [default 4]: ") or 4)
    except:
        n_qubits = 4
        
    print("\nFormat Guidelines:")
    print("  SUM (Two singles):     1^ 0 + 3^ 2")
    print("  PRODUCT (One double):  3^ 2 1^ 0")
    
    user_input = input("Enter Expression: ").strip()
    if not user_input:
        user_input = "1^ 0 + 3^ 2"

    # 2. Parse
    T = robust_parse(user_input, n_qubits)
    if T is None: return

    # 3. Construct Generator G = T - T_dagger
    print("\n--- Constructing Generator ---")
    T_dag = T.adjoint()
    G = (T - T_dag).simplify()
    
    # Remove near-zero terms
    G = SparsePauliOp(G.paulis, np.where(np.abs(G.coeffs) < 1e-10, 0, G.coeffs))
    G = G.simplify()

    # --- NEW: EXPLICITLY PRINT THE FORM OF THE GENERATOR ---
    print("\n" + "="*40)
    print("FORM OF THE GENERATOR (Pauli Sum):")
    print(f"Note: Qiskit ordering is q_{n_qubits-1} ... q_0")
    print("-" * 40)
    
    # Iterate through individual Pauli strings to print them clearly
    for pauli, coeff in zip(G.paulis, G.coeffs):
        # Format complex number 
        c_str = f"{coeff:.2f}"
        print(f"  {c_str:>10} * {pauli}")
    
    print("="*40 + "\n")
    # -------------------------------------------------------
    
    # 4. Check Pairwise Commutivity
    print("Checking commutativity of all internal strings...")
    all_commute = True
    bad_pairs = []
    
    ops_list = [SparsePauliOp([p], [1.0]) for p in G.paulis]
    
    for i, j in combinations(range(len(ops_list)), 2):
        comm = (ops_list[i] @ ops_list[j]) - (ops_list[j] @ ops_list[i])
        comm = comm.simplify()
        
        if not np.all(np.isclose(comm.coeffs, 0.0)):
            all_commute = False
            bad_pairs.append((ops_list[i].paulis[0], ops_list[j].paulis[0]))
            
    # 5. Result
    print("-" * 40)
    if all_commute:
        print("RESULT: ALL TERMS COMMUTE.")
        print("This operator can be split into exact sequential rotations.")
    else:
        print(f"RESULT: FAIL ({len(bad_pairs)} non-commuting pairs found).")
        print("This operator requires Trotterization.")
        print("Bad pairs:", bad_pairs)
    print("-" * 40)

if __name__ == "__main__":
    check_commutativity_of_generator()

Enter system size (qubits) [default 4]:  4



Format Guidelines:
  SUM (Two singles):     1^ 0 + 3^ 2
  PRODUCT (One double):  3^ 2 1^ 0


Enter Expression:  1^ 0



--- Parsing Input: '1^ 0' ---
Structure: SINGLE TERM (Product sequence).
  - Term 1: a_1^ a_0

--- Constructing Generator ---

FORM OF THE GENERATOR (Pauli Sum):
Note: Qiskit ordering is q_3 ... q_0
----------------------------------------
  0.00+0.50j * IIXY
  0.00-0.50j * IIYX

Checking commutativity of all internal strings...
----------------------------------------
RESULT: ALL TERMS COMMUTE.
This operator can be split into exact sequential rotations.
----------------------------------------


In [13]:
%pip list | grep qiskit

qiskit                  2.2.3
qiskit-aer              0.17.2
qiskit-algorithms       0.4.0
qiskit-ibm-runtime      0.44.0
qiskit-nature           0.7.2
Note: you may need to restart the kernel to use updated packages.


<div style="background-color:#f0f0f0; padding:10px; border-radius:5px;">
This code is a part of Quantum AI Biomedical Research Lab project 
    
`Estimating molecular ground and excited state energies on quantum computers'
    
© Copyright Renata Wong, 2026.

This code is licensed under the CC BY-NC 4.0 License. You may
obtain a copy of this license in the LICENSE.txt file in the root directory
of this source tree or at https://creativecommons.org/licenses/by-nc/4.0/deed.en.

Any modifications or derivative works of this code must retain this
copyright notice, and modified files need to carry a notice indicating
that they have been altered from the originals.
</div>

<div style="background-color:#f0f0f0; padding:10px; border-radius:5px;">
This work was supported by the National Science and Technology Council (Taiwan) grant No. NSTC 114-2112-M-182-002-MY3 and Chang Gung Memorial Hospital grant No. BMRPL94.

This work comes with an accompanying paper titled 'Quantum circuit compilation for fermionic excitations using the Jordan-Wigner mapping'.
</div>